# Using Keras on top of TensorFlow for fast prototyping

**Objective**: Build a deep learning model that can learn the alphabet using Keras and compare with TensorFlow implementation

**Agenda**
1. Recap of the previous chapter
2. What is Keras and why should I care?
3. Implementing the same model with Keras
   - Building the computational graph
   - Plugging-in the TensorBoard visualization
   - Training and evaluating the model
4. Conclusions on the difference between TensorFlow and Keras

## References

* [Keras documentation](https://keras.io)